In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")

In [ ]:
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [ ]:
print(df.head())  # Show first 5 rows


   flags                                        instruction category  \
0      B   question about cancelling order {{Order Number}}    ORDER   
1    BQZ  i have a question about cancelling oorder {{Or...    ORDER   
2   BLQZ    i need help cancelling puchase {{Order Number}}    ORDER   
3     BL         I need to cancel purchase {{Order Number}}    ORDER   
4  BCELN  I cannot afford this order, cancel purchase {{...    ORDER   

         intent                                           response  
0  cancel_order  I've understood you have a question regarding ...  
1  cancel_order  I've been informed that you have a question ab...  
2  cancel_order  I can sense that you're seeking assistance wit...  
3  cancel_order  I understood that you need assistance with can...  
4  cancel_order  I'm sensitive to the fact that you're facing f...  


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
    return text

In [ ]:
df['clean_query'] = df['query_text'].apply(clean_text)


In [ ]:
df.describe()

,flags,instruction,category,intent,response
count,26872,26872,26872,26872,26872
unique,394,24635,11,27,26870
top,BL,shiping to {{Delivery City}},ACCOUNT,contact_customer_service,"Firstly, I truly understand how pivotal the {{..."
freq,5212,8,5986,1000,2


In [ ]:
df.shape

(26872, 5)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Drop missing values
df = df.dropna()

# Encode 'category' column (since we are classifying queries)
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# Split data into training and testing sets
X = df['instruction']  # Input: customer query
y = df['category_encoded']  # Output: category label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data Preprocessing Completed!")


Data Preprocessing Completed!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 1.00


In [ ]:
joblib.dump(model, "query_classifier.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [ ]:
def generate_response(user_query):
    # Load saved models
    model = joblib.load("query_classifier.pkl")
    vectorizer = joblib.load("tfidf_vectorizer.pkl")
    label_encoder = joblib.load("label_encoder.pkl")

    # Transform input query
    query_tfidf = vectorizer.transform([user_query])

    # Predict category
    predicted_category_encoded = model.predict(query_tfidf)[0]
    predicted_category = label_encoder.inverse_transform([predicted_category_encoded])[0]

    # Retrieve response
    response = df[df['category'] == predicted_category]['response'].values[0]

    return predicted_category, response

# Example
# user_input = "Where is my order?"
# category, response = generate_response(user_input)
# print(f"Predicted Category: {category}")
# print(f"Response: {response}")

user_input = "Why is my payment failing?"
category, response = generate_response(user_input)
print(f"Predicted Category: {category}")
print(f"Response: {response}")


Predicted Category: PAYMENT
Response: Of course! I appreciate your attempt to access the list of our available payment methods. Allow me to provide you with all the information you need to fulfill your query. Here's a comprehensive list of our accepted payment methods:

- **Credit/Debit Card**: We accept major cards such as Visa, Mastercard, and American Express.
- **PayPal**: A widely used and secure online payment platform.
- **Bank Transfer**: You can directly transfer funds from your bank account to complete the payment.
- **Apple Pay**: For convenient payments through Apple devices.
- **Google Wallet**: A quick and easy payment method for Google users.

I hope this information is helpful to you. If you have any further questions or if there's anything else I can assist you with, please let me know.


In [ ]:
import streamlit as st
import joblib

#



ModuleNotFoundError: No module named 'streamlit'

In [ ]:
pip install streamlit joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import joblib

In [ ]:
# Load saved models
model = joblib.load("query_classifier.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")
label_encoder = joblib.load("label_encoder.pkl")



In [ ]:
def generate_response(user_query):
    query_tfidf = vectorizer.transform([user_query])
    predicted_category_encoded = model.predict(query_tfidf)[0]
    predicted_category = label_encoder.inverse_transform([predicted_category_encoded])[0]

    # Retrieve response
    response = df[df['category'] == predicted_category]['response'].values[0]
    return predicted_category, response


In [ ]:
st.title("AI-Powered Customer Support Assistant")
st.write("Ask any question related to customer support!")

2025-03-29 08:44:28.731 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:44:28.956 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-29 08:44:28.960 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:44:28.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:44:28.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:44:28.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:44:28.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:

# User Input
user_query = st.text_input("I need to cancel purchase {{}}}")

2025-03-29 08:46:12.729 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:46:12.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:46:12.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:46:12.732 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:46:12.733 Session state does not function when running a script without `streamlit run`
2025-03-29 08:46:12.734 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 08:46:12.734 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
!pip install streamlit pyngrok



In [ ]:
with open("app.py", "w") as f:
    f.write("""
import streamlit as st

st.title("AI-Powered Customer Support Chatbot 🤖")
st.write("Ask a question, and I'll try to help!")

user_input = st.text_input("Enter your query here:")
if st.button("Get Response"):
    st.success(f"Response: AI-generated response for '{user_input}'")
""")  # Ensure triple quotes are closed


In [ ]:
!pip install streamlit pyngrok  # Install necessary libraries

from pyngrok import ngrok

!streamlit run app.py &  # Run Streamlit app in the background

public_url = ngrok.connect(port="8501")  # Expose the app using ngrok
print(f"Access your app at: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.149.67:8501

